# UK used Cars - Car Price Prediction Regression Model
--- 

In [2]:
## EDA Standard Libary

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats as ss

In [3]:
import os
#Define ROOT_DIR as the root directory of the project on your local machine
ROOT_DIS = os.path.realpath(os.path.join(os.path.dirname('1-Data Preparation.ipynb'), '..'))
ROOT_DIS


'D:\\Purwadhika\\Final Project\\Brazilian E-Commerce'

## Stakeholder Statements
---

###     Business Problem
Online used car listing company (e.g: OLX Autos, Carmudi, Carsome.id) 

### Business Model: 
Mereka beli mobil bekas dengan kondisi yang bagus harga dibawah pasar, untuk dijual lagi di platform mereka dengan harga yang lebih tinggi

### Business Problem:
- Jika membeli mobil harganya ketinggian maka profitnya lebih kecil

### Business Success Criteria:
- Mereka ingin mengetahui range harga mobil bekas yang akan mereka beli, agar mereka bisa menentukan harga jual yang tepat
- Tetap memperhitungkan profit dalam hasil prediksi harga mobil bekas

### 
Solution:
- Membuat model machine learning untuk memprediksi harga mobil bekas, sehingga ketika agen pembeli mobil bekas menemukan mobil bekas yang akan dibeli, mereka bisa mengetahui range harga beli yang tepat, sehingga profit yang didapat maksimal

print('hello)

### CCLASS (MERCY)

In [4]:
cclass = pd.read_csv("Raw Dataset/cclass.csv")
cclass

,model,year,price,transmission,mileage,fuelType,engineSize
0,C Class,2020,30495,Automatic,1200,Diesel,2.0
1,C Class,2020,29989,Automatic,1000,Petrol,1.5
2,C Class,2020,37899,Automatic,500,Diesel,2.0
3,C Class,2019,30399,Automatic,5000,Diesel,2.0
4,C Class,2019,29899,Automatic,4500,Diesel,2.0
...,...,...,...,...,...,...,...
3894,C Class,2017,14700,Manual,31357,Diesel,1.6
3895,C Class,2018,18500,Automatic,28248,Diesel,2.1
3896,C Class,2014,11900,Manual,48055,Diesel,2.1
3897,C Class,2014,11300,Automatic,49865,Diesel,2.1


In [5]:
cclass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3899 entries, 0 to 3898
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         3899 non-null   object 
 1   year          3899 non-null   int64  
 2   price         3899 non-null   int64  
 3   transmission  3899 non-null   object 
 4   mileage       3899 non-null   int64  
 5   fuelType      3899 non-null   object 
 6   engineSize    3899 non-null   float64
dtypes: float64(1), int64(3), object(3)
memory usage: 213.4+ KB


In [6]:
unclean = pd.read_csv("Raw Dataset/unclean cclass.csv")
unclean

,model,year,price,transmission,mileage,fuel type,engine size,mileage2,fuel type2,engine size2,reference
0,C Class,2020.0,"£30,495",Automatic,NaN,Diesel,2,"1,200",NaN,NaN,/ad/25017331
1,C Class,2020.0,"£29,989",Automatic,NaN,Petrol,1.5,"1,000",NaN,NaN,/ad/25043746
2,C Class,2020.0,"£37,899",Automatic,NaN,Diesel,2,500,NaN,NaN,/ad/25142894
3,C Class,2019.0,"£30,399",Automatic,NaN,Diesel,2,"5,000",NaN,NaN,/ad/24942816
4,C Class,2019.0,"£29,899",Automatic,NaN,Diesel,2,"4,500",NaN,NaN,/ad/24913660
...,...,...,...,...,...,...,...,...,...,...,...
4001,C Class,2017.0,"£14,700",Manual,"31,357",25,£150,70.6,Diesel,1.598,/ad/25451436
4002,C Class,2018.0,"£18,500",Automatic,"28,248",31,£150,64.2,Diesel,2.143,/ad/25451481
4003,C Class,2014.0,"£11,900",Manual,"48,055",31,£20,65.7,Diesel,2.143,/ad/25057204
4004,C Class,2014.0,"£11,300",Automatic,"49,865",46,£145,56.5,Diesel,2.143,/ad/25144481


In [7]:
unclean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4006 entries, 0 to 4005
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         3907 non-null   object 
 1   year          3904 non-null   float64
 2   price         3907 non-null   object 
 3   transmission  3907 non-null   object 
 4   mileage       3808 non-null   object 
 5   fuel type     1329 non-null   object 
 6   engine size   3842 non-null   object 
 7   mileage2      3890 non-null   object 
 8   fuel type2    3808 non-null   object 
 9   engine size2  3808 non-null   object 
 10  reference     3907 non-null   object 
dtypes: float64(1), object(10)
memory usage: 344.4+ KB


Ketika melakukan pengecekan pada Dataset CClass dan Unclean CClass ditemukan bahwa isi data tersebut sama, hanya saja untuk yang CClass itu merupakan versi yang sudah dicleaning

### Checking CClass dan Merc

In [8]:
merc = pd.read_csv("Raw Dataset/merc.csv")
merc

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,SLK,2005,5200,Automatic,63000,Petrol,325,32.1,1.8
1,S Class,2017,34948,Automatic,27000,Hybrid,20,61.4,2.1
2,SL CLASS,2016,49948,Automatic,6200,Petrol,555,28.0,5.5
3,G Class,2016,61948,Automatic,16000,Petrol,325,30.4,4.0
4,G Class,2016,73948,Automatic,4000,Petrol,325,30.1,4.0
...,...,...,...,...,...,...,...,...,...
13114,C Class,2020,35999,Automatic,500,Diesel,145,55.4,2.0
13115,B Class,2020,24699,Automatic,2500,Diesel,145,55.4,2.0
13116,GLC Class,2019,30999,Automatic,11612,Diesel,145,41.5,2.1
13117,CLS Class,2019,37990,Automatic,2426,Diesel,145,45.6,2.0


In [9]:
merc['model'] = merc['model'].str.strip()

In [10]:
merc[merc['model'] == 'C Class']

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
16,C Class,2017,19890,Automatic,30281,Diesel,150,60.1,2.1
19,C Class,2014,15701,Automatic,20498,Diesel,20,64.2,2.1
20,C Class,2019,26601,Automatic,9949,Diesel,150,64.2,2.0
21,C Class,2016,17672,Automatic,31656,Diesel,30,64.2,2.1
29,C Class,2016,19328,Automatic,25042,Diesel,30,64.2,2.1
...,...,...,...,...,...,...,...,...,...
13106,C Class,2019,27999,Automatic,6537,Petrol,145,44.1,2.0
13107,C Class,2019,24999,Automatic,6064,Diesel,145,61.4,2.0
13112,C Class,2019,30399,Automatic,5000,Diesel,145,61.4,2.0
13113,C Class,2020,30999,Automatic,1000,Diesel,145,64.2,2.0


In [11]:
cclass.insert(6, 'tax', 0)
cclass

,model,year,price,transmission,mileage,fuelType,tax,engineSize
0,C Class,2020,30495,Automatic,1200,Diesel,0,2.0
1,C Class,2020,29989,Automatic,1000,Petrol,0,1.5
2,C Class,2020,37899,Automatic,500,Diesel,0,2.0
3,C Class,2019,30399,Automatic,5000,Diesel,0,2.0
4,C Class,2019,29899,Automatic,4500,Diesel,0,2.0
...,...,...,...,...,...,...,...,...
3894,C Class,2017,14700,Manual,31357,Diesel,0,1.6
3895,C Class,2018,18500,Automatic,28248,Diesel,0,2.1
3896,C Class,2014,11900,Manual,48055,Diesel,0,2.1
3897,C Class,2014,11300,Automatic,49865,Diesel,0,2.1


In [12]:
cclass.insert(7, 'mpg', np.nan)
cclass

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,C Class,2020,30495,Automatic,1200,Diesel,0,NaN,2.0
1,C Class,2020,29989,Automatic,1000,Petrol,0,NaN,1.5
2,C Class,2020,37899,Automatic,500,Diesel,0,NaN,2.0
3,C Class,2019,30399,Automatic,5000,Diesel,0,NaN,2.0
4,C Class,2019,29899,Automatic,4500,Diesel,0,NaN,2.0
...,...,...,...,...,...,...,...,...,...
3894,C Class,2017,14700,Manual,31357,Diesel,0,NaN,1.6
3895,C Class,2018,18500,Automatic,28248,Diesel,0,NaN,2.1
3896,C Class,2014,11900,Manual,48055,Diesel,0,NaN,2.1
3897,C Class,2014,11300,Automatic,49865,Diesel,0,NaN,2.1


In [13]:
cclass

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,C Class,2020,30495,Automatic,1200,Diesel,0,NaN,2.0
1,C Class,2020,29989,Automatic,1000,Petrol,0,NaN,1.5
2,C Class,2020,37899,Automatic,500,Diesel,0,NaN,2.0
3,C Class,2019,30399,Automatic,5000,Diesel,0,NaN,2.0
4,C Class,2019,29899,Automatic,4500,Diesel,0,NaN,2.0
...,...,...,...,...,...,...,...,...,...
3894,C Class,2017,14700,Manual,31357,Diesel,0,NaN,1.6
3895,C Class,2018,18500,Automatic,28248,Diesel,0,NaN,2.1
3896,C Class,2014,11900,Manual,48055,Diesel,0,NaN,2.1
3897,C Class,2014,11300,Automatic,49865,Diesel,0,NaN,2.1


In [14]:
merc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13119 entries, 0 to 13118
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         13119 non-null  object 
 1   year          13119 non-null  int64  
 2   price         13119 non-null  int64  
 3   transmission  13119 non-null  object 
 4   mileage       13119 non-null  int64  
 5   fuelType      13119 non-null  object 
 6   tax           13119 non-null  int64  
 7   mpg           13119 non-null  float64
 8   engineSize    13119 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 922.6+ KB


In [15]:
cclass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3899 entries, 0 to 3898
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         3899 non-null   object 
 1   year          3899 non-null   int64  
 2   price         3899 non-null   int64  
 3   transmission  3899 non-null   object 
 4   mileage       3899 non-null   int64  
 5   fuelType      3899 non-null   object 
 6   tax           3899 non-null   int64  
 7   mpg           0 non-null      float64
 8   engineSize    3899 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 274.3+ KB


In [16]:
mercedes = merc.merge(cclass, how = 'outer')

In [17]:
mercedes['model'].value_counts()

model
 C Class     3899
C Class      3747
A Class      2561
E Class      1953
GLC Class     960
GLA Class     847
B Class       591
CL Class      511
GLE Class     461
SL CLASS      260
CLS Class     237
V Class       207
S Class       197
GL Class      121
SLK            95
CLA Class      86
X-CLASS        82
M Class        79
GLS Class      74
GLB Class      19
G Class        15
CLK             7
CLC Class       3
R Class         2
180             1
230             1
220             1
200             1
Name: count, dtype: int64

In [22]:
mercedes['model'] = mercedes['model'].str.strip()
c_class_data = mercedes[mercedes['model'] == 'C Class']

def replace_zeros(series):
    return series.replace(0, method='ffill')

c_class_data['tax'] = c_class_data.groupby('year')['tax'].transform(replace_zeros)

mercedes.update(c_class_data)
mercedes

C:\Users\USER-PC\AppData\Local\Temp\ipykernel_21256\3337801954.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_class_data['tax'] = c_class_data.groupby('year')['tax'].transform(replace_zeros)


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,SLK,2005,5200,Automatic,63000,Petrol,325,32.1,1.8
1,S Class,2017,34948,Automatic,27000,Hybrid,20,61.4,2.1
2,SL CLASS,2016,49948,Automatic,6200,Petrol,555,28.0,5.5
3,G Class,2016,61948,Automatic,16000,Petrol,325,30.4,4.0
4,G Class,2016,73948,Automatic,4000,Petrol,325,30.1,4.0
...,...,...,...,...,...,...,...,...,...
17013,C Class,2017,14700,Manual,31357,Diesel,150,NaN,1.6
17014,C Class,2018,18500,Automatic,28248,Diesel,145,NaN,2.1
17015,C Class,2014,11900,Manual,48055,Diesel,20,NaN,2.1
17016,C Class,2014,11300,Automatic,49865,Diesel,20,NaN,2.1


In [26]:
mercedes[mercedes['tax'] == 0]


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
512,A Class,2015,11498,Manual,32648,Diesel,0,74.3,1.5
917,A Class,2016,13695,Manual,41600,Diesel,0,80.7,1.5
1131,B Class,2014,8200,Manual,39331,Diesel,0,74.3,1.5
1546,A Class,2014,10490,Manual,37642,Diesel,0,78.5,1.5
1579,A Class,2015,10490,Semi-Auto,49816,Diesel,0,70.6,1.5
...,...,...,...,...,...,...,...,...,...
12626,A Class,2016,11400,Manual,44277,Diesel,0,80.7,1.5
16608,C Class,1991,4450,Automatic,170000,Petrol,0,NaN,3.0
16656,C Class,1995,4750,Automatic,173000,Petrol,0,NaN,3.6
16752,C Class,2003,1795,Automatic,88082,Diesel,0,NaN,2.1


In [28]:
mercedes.drop(mercedes[mercedes['tax'] == 0].index, inplace=True)

In [30]:
mercedes

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,SLK,2005,5200,Automatic,63000,Petrol,325,32.1,1.8
1,S Class,2017,34948,Automatic,27000,Hybrid,20,61.4,2.1
2,SL CLASS,2016,49948,Automatic,6200,Petrol,555,28.0,5.5
3,G Class,2016,61948,Automatic,16000,Petrol,325,30.4,4.0
4,G Class,2016,73948,Automatic,4000,Petrol,325,30.1,4.0
...,...,...,...,...,...,...,...,...,...
17013,C Class,2017,14700,Manual,31357,Diesel,150,NaN,1.6
17014,C Class,2018,18500,Automatic,28248,Diesel,145,NaN,2.1
17015,C Class,2014,11900,Manual,48055,Diesel,20,NaN,2.1
17016,C Class,2014,11300,Automatic,49865,Diesel,20,NaN,2.1


In [33]:
median_mpg_c_class = c_class_data['mpg'].median()

mercedes.loc[mercedes['model'] == 'C Class', 'mpg'] = mercedes.loc[mercedes['model'] == 'C Class', 'mpg'].fillna(median_mpg_c_class)
mercedes

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,SLK,2005,5200,Automatic,63000,Petrol,325,32.1,1.8
1,S Class,2017,34948,Automatic,27000,Hybrid,20,61.4,2.1
2,SL CLASS,2016,49948,Automatic,6200,Petrol,555,28.0,5.5
3,G Class,2016,61948,Automatic,16000,Petrol,325,30.4,4.0
4,G Class,2016,73948,Automatic,4000,Petrol,325,30.1,4.0
...,...,...,...,...,...,...,...,...,...
17013,C Class,2017,14700,Manual,31357,Diesel,150,57.7,1.6
17014,C Class,2018,18500,Automatic,28248,Diesel,145,57.7,2.1
17015,C Class,2014,11900,Manual,48055,Diesel,20,57.7,2.1
17016,C Class,2014,11300,Automatic,49865,Diesel,20,57.7,2.1
